In [8]:
# General packages                                                                                                                                                                                                                                                                                                                                                                                                                        import pandas as pd 
import numpy as np
import pandas as pd
import requests
import time

# BeautifulSoup
from bs4 import BeautifulSoup

# Selenium packages
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains


In [9]:
# Open tab in full screen (maximized)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument("start-maximized")

In [13]:
# Instantiate Selenium Webdriver
driver = webdriver.Chrome(options = chrome_options , executable_path = r'C:\Users\Deepthi Poojary\Web Scrapping\chromedriver')

# Get the main page (Top Restaurants in Mumbai on Zomato)
driver.get("https://www.zomato.com/mumbai/great-food-no-bull")

# Get page soure
source = driver.page_source

In [11]:
# Get the main page/window handle after it has opened
main_window = driver.window_handles[0]

In [12]:
# Get title of  main window
main_window_title = driver.title
print(main_window_title)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=80.0.3987.106)


In [ ]:
# Calculating number of restaurant on the main page using BeautifulSoup and main page source
soup = BeautifulSoup(source,"html.parser") # col-s-8 col-l-1by3
type(soup)
button = soup.find_all("div", attrs = {"class":"col-s-8 col-l-1by3"})
bs_len = len(button)
bs_len

This is the number of restaurant currently on the main page.

In [ ]:
# Calculating number of restaurants on the main page using Selenium's find_elements_by_css_selector 
links = driver.find_elements_by_css_selector("div[class = 'col-s-8 col-l-1by3']")
sel_len = len(links)
sel_len

This is just to confirm the number of restaurants on main page and hence bs_len should be equal to sel_len.

In [ ]:
type(links)

In [ ]:
#  Data we'll be extracting
# Made a dictionary for ease of structuring dataframe
table = {"Name":[],"Area":[],"Type":[],"Rating":[],"Ph_Number":[],"Cuisine":[],"Avg_price":[],"Beer":[],"Time":[],"Adress":[],"Facility":[],"Food_Rating":[],"Service_Rating":[],"Ambience_Rating":[] }

In [ ]:
def subs(val):
    if val is None:
        val = np.NAN
    return val

In [ ]:
for link in links:        # for loop loops through links of all restaurants in links variable

    
    ActionChains(driver).key_down(Keys.CONTROL).click(link).key_up(Keys.CONTROL).perform()        # Opens each restaurants link in new tab by  "ctrl" + clicking (link) 
    
    # get the window handle after a new window/tab has opened
    new_window = driver.window_handles[1]
    
    # Switch focus to new window so that required operations can be done in that tab
    driver.switch_to.window(new_window)
    
    # get new window title
    new_window_title = driver.title
    
    if main_window_title != new_window_title:  # Check if main_window and new_window are different
                    
        
        # Providing page source of new window
        Source = driver.page_source
        
        

        # BeautifulSoup 
        soup1 = BeautifulSoup(Source, "html.parser")
        outlet = driver.find_elements_by_css_selector("div[class = 'card  search-snippet-card     search-card  ']") 
##### For Resaturant having more than one outlet
        if outlet:
            optional = driver.find_elements_by_css_selector("div[class= 'search_left_featured clearfix']")
            lh = range(len(optional))      
            for l in lh:       # iterates for each outlet
                # Opening multi outlet restaurant's each outlet in 3rd tab 
                ActionChains(driver).key_down(Keys.CONTROL).click(optional[l]).key_up(Keys.CONTROL).perform()  
                WebDriverWait(driver,70).until(EC.number_of_windows_to_be(3))
                #l.click()       # Clicking on the link and opening on the same page
                newest_handle = driver.window_handles[2]   # Window handle starts with 0, for third window/tab handle no. will 2
                driver.switch_to.window(newest_handle)
                SourceXL = driver.page_source
                soup2 = BeautifulSoup(SourceXL,'html.parser')


                ### Gate 1
                header = soup2.find("div", attrs =  {"class" : "col-l-12"})
                # Restaurant Name
                Name1 = soup2.find("div", attrs = {"class" : "col-l-12"})
                if Name1 is not None:                                         # Check if a particular data exist
                    Name = Name1.h1.text.strip('\n').strip(" ")
                else:
                    Name = subs(Name1)                                        # Use subs function to substitute NAN if a data doesn't exist
                table["Name"].append(Name)
                # Main adress
                Area1 = header
                if Area1 is not None:
                    Area = Area1.div.a.text.strip(" ").strip("\n").strip(" ")
                else:
                    Area = subs(Area1)
                table["Area"].append(Area)
                # Type of Restaurant
                Type1 = header.find("span", attrs = {"class" : "res-info-estabs grey-text fontsize3"})
                if Type1 is not None:
                    Type = Type1.text.strip("\n")
                else:
                    Type = subs(Type1)
                table["Type"].append(Type)


                ### Gate 2
                header2 = soup2.find("div"  , attrs = {"class" : "col-l-4 tac left"})
                #Rating of restaurent
                Rating1 = header2.find("div" , attrs = {"tabindex" : "0"})
                if Rating1 is not None:
                    Rating = Rating1.text.strip().replace(" ","").replace("\n","")
                else:
                    Rating = subs(Rating1)
                table["Rating"].append(Rating)


                ### Gate 3
                tail1 = soup2.find("div" , class_ = "col-l-1by3 pl0 pr20")
                #Phone Number 
                Ph_Number1 = tail1.find("span" , attrs = {"class" : "fontsize2 bold zgreen"})
                if Ph_Number1 is not None:
                    Ph_Number = Ph_Number1.text
                else:
                    Ph_Number = subs(Ph_Number1)
                table["Ph_Number"].append(Ph_Number)
                #Cusines
                Cuisine1 = tail1.find("div" , class_ = "res-info-cuisines clearfix")
                if Cuisine1 is not None:
                    Cuisine = Cuisine1.text
                else:
                    Cuisine = subs(Cuisine1)
                table["Cuisine"].append(Cuisine)
                #Average  price or price for two person
                Avg_price1 = tail1.find("div" , class_ = "res-info-detail").find("span" ,attrs = {"tabindex" : "0"})
                if Avg_price1 is not None:
                    Avg_price = Avg_price1.text.split("for")[0]
                else:
                    Avg_price = subs(Avg_price1)
                table["Avg_price"].append(Avg_price)
                #Beer price/pint
                Beer1 = tail1.find("div" , class_ = "res-info-detail").find("div",class_ = "mt5")
                if Beer1 is not None:
                    Beer = Beer1.text.strip("\n").split(" ")[0]
                else:
                    Beer = subs(Beer1)
                table["Beer"].append(Beer)


                ### Gate 4
                tail2= soup2.find("div",attrs={"class" : "col-l-1by3 pl20 pr20"})
                # Today's timing
                Time1 = tail2.find("div",attrs= {"class":"clearfix"}).find("div",class_ = "medium")
                if Time1 is not None:
                    Time = Time1.text.strip("Today")
                else:
                    Time = subs(Time1)
                table["Time"].append(Time)
                # Full Adress
                Address1 = tail2.find("div" , class_ = "resinfo-icon")
                if Address1 is not None:
                    Address = Address1.span.text
                else:
                    Address = subs(Address1)
                table["Adress"].append(Address)

                # Using "What people love here" to get food rating, service rating and ambience rating.
                ### Gate 5
                tail_end = soup2.find_all("div" , attrs = {"class" , "col-l-1by3 pl20 pr20"})[1]
                #Additional_Info
                Facility1 = tail_end.find("div",attrs={"class" : "res-info-highlights"})
                if Facility1 is not None:
                    Facility = Facility1.text.strip("\n").replace("\n"," ").replace("More Info","").strip()
                else:
                    Facility = subs(Facility1)
                table["Facility"].append(Facility)
                # Food rating
                exp = soup2.find_all("div" , class_ = "rv_highlights__score_bar mt5 mb5")
                length = len(exp)
                if length >=1:       # Some restaurant dont have data for any of the rating

                    lis1 = exp[0].find_all("div" , class_ = "block level-8")
                    if lis1 is not None:
                        n1 = 0
                        for element in lis1:
                            n1 += 1
                        food_rating = n1
                    else:
                        food_rating = subs(food_rating)
                    table["Food_Rating"].append(food_rating)

                    if length >= 2:
                        # Service rating
                        lis2 = exp[1].find_all("div" , class_ = "block level-7")
                        if lis2 is not None:
                            n2 = 0
                            for element in lis2:
                                n2 += 1
                            service_rating = n2
                        else:
                            service_rating = subs(service_rating)
                        table["Service_Rating"].append(service_rating)



                        if length >= 3:       # Some restaurant don't have data for ambience rating

                                # Ambience Rating
                            lis3 = exp[2].find_all("div" , class_ = "block level-6")
                            if lis3 is not None:
                                n3 = 0
                                for element in lis3:
                                    n3+=1
                                ambience_rating = n3
                            else:
                                ambience_rating = subs(ambience_rating)
                            table["Ambience_Rating"].append(ambience_rating)
                        else:
                            hue = np.NAN
                            table["Ambience_Rating"].append(hue)
                    else:
                        hue = np.NAN
                        table["Service_Rating"].append(hue)
                        table["Ambience_Rating"].append(hue)
                else:
                    hue = np.NAN
                    table["Food_Rating"].append(hue)
                    table["Service_Rating"].append(hue)
                    table["Ambience_Rating"].append(hue)
        #                     else:              # if no ambience rating put NA 
        #                         hue =np.NAN
        #                         table["Ambience_Rating"].append(hue)
#                      #Ask Selenium to click the back button
#                 else:
#                     na = np.NAN
#                     table["Service_Rating"].append(na)
#                     table["Ambience_Rating"].append(na)
                    
                    #drive.execute_script("window.history.go(-1)") 
                driver.close()
                #driver.execute_script("window.history.go(-1)")
                driver.switch_to.window(new_window)
                
        else:
            ### Gate 1
                header = soup1.find("div", attrs =  {"class" : "col-l-12"})
                # Restaurant Name
                Name1 = soup1.find("div", attrs = {"class" : "col-l-12"})
                if Name1 is not None:
                    Name = Name1.h1.text.strip('\n').strip(" ")
                else:
                    Name = subs(Name1)
                table["Name"].append(Name)
                # Main adress
                Area1 = header
                if Area1 is not None:
                    Area = Area1.div.a.text.strip(" ").strip("\n").strip(" ")
                else:
                    Area = subs(Area1)
                table["Area"].append(Area)
                # Type of Restaurant
                Type1 = header.find("span", attrs = {"class" : "res-info-estabs grey-text fontsize3"})
                if Type1 is not None:
                    Type = Type1.text.strip("\n")
                else:
                    Type = subs(Type1)
                table["Type"].append(Type)


                ### Gate 2
                header2 = soup1.find("div"  , attrs = {"class" : "col-l-4 tac left"})
                #Rating of restaurent
                Rating1 = header2.find("div" , attrs = {"tabindex" : "0"})
                if Rating1 is not None:
                    Rating = Rating1.text.strip().replace(" ","").replace("\n","")
                else:
                    Rating = subs(Rating1)
                table["Rating"].append(Rating)


                ### Gate 3
                tail1 = soup1.find("div" , class_ = "col-l-1by3 pl0 pr20")
                #Phone Number 
                Ph_Number1 = tail1.find("span" , attrs = {"class" : "fontsize2 bold zgreen"})
                if Ph_Number1 is not None:
                    Ph_Number = Ph_Number1.text
                else:
                    Ph_Number = subs(Ph_Number1)
                table["Ph_Number"].append(Ph_Number)
                #Cusines
                Cuisine1 = tail1.find("div" , class_ = "res-info-cuisines clearfix")
                if Cuisine1 is not None:
                    Cuisine = Cuisine1.text
                else:
                    Cuisine = subs(Cuisine1)
                table["Cuisine"].append(Cuisine)
                #Average  price or price for two person
                Avg_price1 = tail1.find("div" , class_ = "res-info-detail").find("span" ,attrs = {"tabindex" : "0"})
                if Avg_price1 is not None:
                    Avg_price = Avg_price1.text.split("for")[0]
                else:
                    Avg_price = subs(Avg_price1)
                table["Avg_price"].append(Avg_price)
                #Beer price/pint
                Beer1 = tail1.find("div" , class_ = "res-info-detail").find("div",class_ = "mt5")
                if Beer1 is not None:
                    Beer = Beer1.text.strip("\n").split(" ")[0]
                else:
                    Beer = subs(Beer1)
                table["Beer"].append(Beer)


                ### Gate 4
                tail2= soup1.find("div",attrs={"class" : "col-l-1by3 pl20 pr20"})
                # Today's timing
                Time1 = tail2.find("div",attrs= {"class":"clearfix"}).find("div",class_ = "medium")
                if Time1 is not None:
                    Time = Time1.text.strip("Today")
                else:
                    Time = subs(Time1)
                table["Time"].append(Time)
                # Full Adress
                Address1 = tail2.find("div" , class_ = "resinfo-icon")
                if Address1 is not None:
                    Address = Address1.span.text
                else:
                    Address = subs(Address1)
                table["Adress"].append(Address)

                # Using "What people love here" to get food rating, service rating and ambience rating.
                ### Gate 5
                tail_end = soup1.find_all("div" , attrs = {"class" , "col-l-1by3 pl20 pr20"})[1]
                #Additional_Info
                Facility1 = tail_end.find("div",attrs={"class" : "res-info-highlights"})
                if Facility1 is not None:
                    Facility = Facility1.text.strip("\n").replace("\n"," ").replace("More Info","").strip()
                else:
                    Facility = subs(Facility1)
                table["Facility"].append(Facility)
                # Food rating
                exp = soup1.find_all("div" , class_ = "rv_highlights__score_bar mt5 mb5")
                length = len(exp)
                if length >=1:

                    lis1 = exp[0].find_all("div" , class_ = "block level-8")
                    if lis1 is not None:
                        n1 = 0
                        for element in lis1:
                            n1 += 1
                        food_rating = n1
                    else:
                        food_rating = subs(food_rating)
                    table["Food_Rating"].append(food_rating)

                    if length >= 2:
                        # Service rating
                        lis2 = exp[1].find_all("div" , class_ = "block level-7")
                        if lis2 is not None:
                            n2 = 0
                            for element in lis2:
                                n2 += 1
                            service_rating = n2
                        else:
                            service_rating = subs(service_rating)
                        table["Service_Rating"].append(service_rating)



                        if length >= 3:       # Some restaurant don't have data for ambience rating

                                # Ambience Rating
                            lis3 = exp[2].find_all("div" , class_ = "block level-6")
                            if lis3 is not None:
                                n3 = 0
                                for element in lis3:
                                    n3+=1
                                ambience_rating = n3
                            else:
                                ambience_rating = subs(ambience_rating)
                            table["Ambience_Rating"].append(ambience_rating)
                            
                        else:
                            hue = np.NAN
                            table["Ambience_Rating"].append(hue)
                    else:
                        hue = np.NAN
                        table["Service_Rating"].append(hue)
                        table["Ambience_Rating"].append(hue)
                else:
                    hue = np.NAN
                    table["Food_Rating"].append(hue)
                    table["Service_Rating"].append(hue)
                    table["Ambience_Rating"].append(hue)
                                 #Ask Selenium to click the back button
                                #drive.execute_script("window.history.go(-1)") 
                
        
        

            # Close new tab after getting all data
        driver.close()


            # Switch handle back to main tab to open next restaurant's link
        driver.switch_to.window(main_window)

In [ ]:
# take a look at dictionary (including 14 sample of restaurant data)
table

In [ ]:
# Display in Dataframe format
dataset = pd.DataFrame(table)
dataset

In [ ]:
# Save it in CSV format
dataset.to_csv("Zomato_dataset.csv")